Before we get started, a couple of reminders to keep in mind when using iPython notebooks:

- Remember that you can see from the left side of a code cell when it was last run if there is a number within the brackets.
- When you start a new notebook session, make sure you run all of the cells up to the point where you last left off. Even if the output is still visible from when you ran the cells in your previous session, the kernel starts in a fresh state so you'll need to reload the data, etc. on a new session.
- The previous point is useful to keep in mind if your answers do not match what is expected in the lesson's quizzes. Try reloading the data and run all of the processing steps one by one in order to make sure that you are working with the same variables and data that are at each quiz stage.


## Load Data from CSVs

In [47]:
import unicodecsv

## Longer version of code (replaced with shorter, equivalent version below)

# enrollments = []
# f = open('enrollments.csv', 'rb')
# reader = unicodecsv.DictReader(f)
# for row in reader:
#     enrollments.append(row)
# f.close()

def read_csv(filename):
    with open(filename, 'rb') as f:
        reader = unicodecsv.DictReader(f)
        return list(reader)
enrollments = read_csv('enrollments.csv')


In [48]:
#####################################
#                 1                 #
#####################################

## Read in the data from daily_engagement.csv and project_submissions.csv 
## and store the results in the below variables.
## Then look at the first row of each table.

daily_engagement = read_csv('daily_engagement.csv')
project_submissions = read_csv('project_submissions.csv')


In [49]:
print len(daily_engagement)
print len(project_submissions)

print enrollments[0]
print
print daily_engagement[0]
print
print project_submissions[0]

136240
3642
{u'status': u'canceled', u'is_udacity': u'True', u'is_canceled': u'True', u'join_date': u'2014-11-10', u'account_key': u'448', u'cancel_date': u'2015-01-14', u'days_to_cancel': u'65'}

{u'lessons_completed': u'0.0', u'num_courses_visited': u'1.0', u'total_minutes_visited': u'11.6793745', u'projects_completed': u'0.0', u'acct': u'0', u'utc_date': u'2015-01-09'}

{u'lesson_key': u'3176718735', u'processing_state': u'EVALUATED', u'account_key': u'256', u'assigned_rating': u'UNGRADED', u'completion_date': u'2015-01-16', u'creation_date': u'2015-01-14'}


## Fixing Data Types

In [50]:
from datetime import datetime as dt

# Takes a date as a string, and returns a Python datetime object. 
# If there is no date given, returns None
def parse_date(date):
    if date == '':
        return None
    else:
        return dt.strptime(date, '%Y-%m-%d')
    
# Takes a string which is either an empty string or represents an integer,
# and returns an int or None.
def parse_maybe_int(i):
    if i == '':
        return None
    else:
        return int(i)

# Clean up the data types in the enrollments table
for enrollment in enrollments:
    enrollment['cancel_date'] = parse_date(enrollment['cancel_date'])
    enrollment['days_to_cancel'] = parse_maybe_int(enrollment['days_to_cancel'])
    enrollment['is_canceled'] = enrollment['is_canceled'] == 'True'
    enrollment['is_udacity'] = enrollment['is_udacity'] == 'True'
    enrollment['join_date'] = parse_date(enrollment['join_date'])
    
enrollments[0]

{u'account_key': u'448',
 u'cancel_date': datetime.datetime(2015, 1, 14, 0, 0),
 u'days_to_cancel': 65,
 u'is_canceled': True,
 u'is_udacity': True,
 u'join_date': datetime.datetime(2014, 11, 10, 0, 0),
 u'status': u'canceled'}

In [51]:
# Clean up the data types in the engagement table
for engagement_record in daily_engagement:
    engagement_record['lessons_completed'] = int(float(engagement_record['lessons_completed']))
    engagement_record['num_courses_visited'] = int(float(engagement_record['num_courses_visited']))
    engagement_record['projects_completed'] = int(float(engagement_record['projects_completed']))
    engagement_record['total_minutes_visited'] = float(engagement_record['total_minutes_visited'])
    engagement_record['utc_date'] = parse_date(engagement_record['utc_date'])
    
daily_engagement[0]

{u'acct': u'0',
 u'lessons_completed': 0,
 u'num_courses_visited': 1,
 u'projects_completed': 0,
 u'total_minutes_visited': 11.6793745,
 u'utc_date': datetime.datetime(2015, 1, 9, 0, 0)}

In [52]:
# Clean up the data types in the submissions table
for submission in project_submissions:
    submission['completion_date'] = parse_date(submission['completion_date'])
    submission['creation_date'] = parse_date(submission['creation_date'])

project_submissions[0]

{u'account_key': u'256',
 u'assigned_rating': u'UNGRADED',
 u'completion_date': datetime.datetime(2015, 1, 16, 0, 0),
 u'creation_date': datetime.datetime(2015, 1, 14, 0, 0),
 u'lesson_key': u'3176718735',
 u'processing_state': u'EVALUATED'}

Note when running the above cells that we are actively changing the contents of our data variables. If you try to run these cells multiple times in the same session, an error will occur.

## Investigating the Data

In [53]:
#####################################
#                 2                 #
#####################################

## Find the total number of rows and the number of unique students (account keys)
## in each table.

### For each of these three tables, find the number of rows in the table and
### the number of unique students in the table. To find the number of unique
### students, you might want to create a set of the account keys in each table.

def createAccountSet(listOfDict, accountKeyName='account_key'):
    accountSet = set()
    for entry in listOfDict:
        accountSet.add(entry[accountKeyName])
    return accountSet

enrollment_num_rows = len(enrollments)             
enrollment_num_unique_students = len(createAccountSet(enrollments))

engagement_num_rows = len(daily_engagement)           
engagement_num_unique_students = len(createAccountSet(daily_engagement, 'acct'))

submission_num_rows = len(project_submissions)             
submission_num_unique_students = len(createAccountSet(project_submissions))

print enrollment_num_rows
print engagement_num_rows
print submission_num_rows
print enrollment_num_unique_students
print engagement_num_unique_students
print submission_num_unique_students

1640
136240
3642
1302
1237
743


In [54]:
from prettytable import PrettyTable

def showTableFromDict(dict): 
    table = PrettyTable(['key','value'])
    for key,val in dict.items():
        table.add_row([key, str(val)])
    print table

showTableFromDict(enrollments[0])
showTableFromDict(daily_engagement[213])
showTableFromDict(project_submissions[0])

+----------------+---------------------+
|      key       |        value        |
+----------------+---------------------+
|     status     |       canceled      |
|   is_udacity   |         True        |
|  is_canceled   |         True        |
|   join_date    | 2014-11-10 00:00:00 |
|  account_key   |         448         |
|  cancel_date   | 2015-01-14 00:00:00 |
| days_to_cancel |          65         |
+----------------+---------------------+
+-----------------------+---------------------+
|          key          |        value        |
+-----------------------+---------------------+
|   lessons_completed   |          0          |
|  num_courses_visited  |          0          |
| total_minutes_visited |         0.0         |
|   projects_completed  |          0          |
|          acct         |          0          |
|        utc_date       | 2015-08-11 00:00:00 |
+-----------------------+---------------------+
+------------------+---------------------+
|       key        |      

## Problems in the Data

In [55]:
#####################################
#                 3                 #
#####################################

## Rename the "acct" column in the daily_engagement table to "account_key".
for entry in daily_engagement:
    value = entry.pop('acct')
    entry['account_key'] = value
    
showTableFromDict(daily_engagement[0])

daily_engagement[0]['account_key']

+-----------------------+---------------------+
|          key          |        value        |
+-----------------------+---------------------+
|   lessons_completed   |          0          |
|  num_courses_visited  |          1          |
| total_minutes_visited |      11.6793745     |
|   projects_completed  |          0          |
|      account_key      |          0          |
|        utc_date       | 2015-01-09 00:00:00 |
+-----------------------+---------------------+


u'0'

## Missing Engagement Records

In [66]:
#####################################
#                 4                 #
#####################################

## Find any one student enrollments where the student is missing from the daily engagement table.
## Output that enrollment.
enrollment_unique_students = createAccountSet(enrollments)
engagement_unique_students = createAccountSet(daily_engagement)
for enroll_record in enrollment_unique_students:
    if False == (enroll_record in engagement_unique_students):
        missing = enroll_record
        break
        
print missing

1145


In [68]:
for enrolls in enrollments:
    if enrolls['account_key'] == missing:
        showTableFromDict(enrolls)
        break

+----------------+---------------------+
|      key       |        value        |
+----------------+---------------------+
|     status     |       canceled      |
|   is_udacity   |        False        |
|  is_canceled   |         True        |
|   join_date    | 2015-04-04 00:00:00 |
|  account_key   |         1145        |
|  cancel_date   | 2015-04-04 00:00:00 |
| days_to_cancel |          0          |
+----------------+---------------------+


- is_udacity is false. 
- join_date and cancel_date are exactly same.
- days to cancel is 0. 

## Checking for More Problem Records

In [72]:
#####################################
#                 5                 #
#####################################

## Find the number of surprising data points (enrollments missing from
## the engagement table) that remain, if any.
missing = []
for enroll in enrollments:
    if False == (enroll['account_key'] in engagement_unique_students):
        if enroll['days_to_cancel'] != 0:
            missing.append(enroll)
len(missing)
        

3

In [73]:
for missing_enroll in missing:
    showTableFromDict(missing_enroll)

+----------------+---------------------+
|      key       |        value        |
+----------------+---------------------+
|     status     |       canceled      |
|   is_udacity   |         True        |
|  is_canceled   |         True        |
|   join_date    | 2015-01-10 00:00:00 |
|  account_key   |         1304        |
|  cancel_date   | 2015-03-10 00:00:00 |
| days_to_cancel |          59         |
+----------------+---------------------+
+----------------+---------------------+
|      key       |        value        |
+----------------+---------------------+
|     status     |       canceled      |
|   is_udacity   |         True        |
|  is_canceled   |         True        |
|   join_date    | 2015-03-10 00:00:00 |
|  account_key   |         1304        |
|  cancel_date   | 2015-06-17 00:00:00 |
| days_to_cancel |          99         |
+----------------+---------------------+
+----------------+---------------------+
|      key       |        value        |
+---------------

According to Udacity data scientist, if 'is_udacity' == True, it is a testing account. Do not use them!

## Tracking Down the Remaining Problems

In [74]:
# Create a set of the account keys for all Udacity test accounts
udacity_test_accounts = set()
for enrollment in enrollments:
    if enrollment['is_udacity']:
        udacity_test_accounts.add(enrollment['account_key'])
len(udacity_test_accounts)

6

In [75]:
# Given some data with an account_key field, removes any records corresponding to Udacity test accounts
def remove_udacity_accounts(data):
    non_udacity_data = []
    for data_point in data:
        if data_point['account_key'] not in udacity_test_accounts:
            non_udacity_data.append(data_point)
    return non_udacity_data

In [76]:
# Remove Udacity test accounts from all three tables
non_udacity_enrollments = remove_udacity_accounts(enrollments)
non_udacity_engagement = remove_udacity_accounts(daily_engagement)
non_udacity_submissions = remove_udacity_accounts(project_submissions)

print len(non_udacity_enrollments)
print len(non_udacity_engagement)
print len(non_udacity_submissions)

1622
135656
3634


## Refining the Question

In [80]:
#####################################
#                 6                 #
#####################################

## Create a dictionary named paid_students containing all students who either
## haven't canceled yet or who remained enrolled for more than 7 days. The keys
## should be account keys, and the values should be the date the student enrolled.

paid_students = {}
for student in non_udacity_enrollments:
    if student['days_to_cancel'] == None or student['days_to_cancel'] > 7:
        paid_students[student['account_key']] = student['join_date']

len(paid_students)

995

## Getting Data from First Week

In [ ]:
# Takes a student's join date and the date of a specific engagement record,
# and returns True if that engagement record happened within one week
# of the student joining.
def within_one_week(join_date, engagement_date):
    time_delta = engagement_date - join_date
    return time_delta.days < 7

In [ ]:
#####################################
#                 7                 #
#####################################

## Create a list of rows from the engagement table including only rows where
## the student is one of the paid students you just found, and the date is within
## one week of the student's join date.

paid_engagement_in_first_week = 

## Exploring Student Engagement

In [ ]:
from collections import defaultdict

# Create a dictionary of engagement grouped by student.
# The keys are account keys, and the values are lists of engagement records.
engagement_by_account = defaultdict(list)
for engagement_record in paid_engagement_in_first_week:
    account_key = engagement_record['account_key']
    engagement_by_account[account_key].append(engagement_record)

In [ ]:
# Create a dictionary with the total minutes each student spent in the classroom during the first week.
# The keys are account keys, and the values are numbers (total minutes)
total_minutes_by_account = {}
for account_key, engagement_for_student in engagement_by_account.items():
    total_minutes = 0
    for engagement_record in engagement_for_student:
        total_minutes += engagement_record['total_minutes_visited']
    total_minutes_by_account[account_key] = total_minutes

In [ ]:
import numpy as np

# Summarize the data about minutes spent in the classroom
total_minutes = total_minutes_by_account.values()
print 'Mean:', np.mean(total_minutes)
print 'Standard deviation:', np.std(total_minutes)
print 'Minimum:', np.min(total_minutes)
print 'Maximum:', np.max(total_minutes)

## Debugging Data Analysis Code

In [ ]:
#####################################
#                 8                 #
#####################################

## Go through a similar process as before to see if there is a problem.
## Locate at least one surprising piece of data, output it, and take a look at it.

## Lessons Completed in First Week

In [ ]:
#####################################
#                 9                 #
#####################################

## Adapt the code above to find the mean, standard deviation, minimum, and maximum for
## the number of lessons completed by each student during the first week. Try creating
## one or more functions to re-use the code above.

## Number of Visits in First Week

In [ ]:
######################################
#                 10                 #
######################################

## Find the mean, standard deviation, minimum, and maximum for the number of
## days each student visits the classroom during the first week.

## Splitting out Passing Students

In [ ]:
######################################
#                 11                 #
######################################

## Create two lists of engagement data for paid students in the first week.
## The first list should contain data for students who eventually pass the
## subway project, and the second list should contain data for students
## who do not.

subway_project_lesson_keys = ['746169184', '3176718735']

passing_engagement =
non_passing_engagement =

## Comparing the Two Student Groups

In [ ]:
######################################
#                 12                 #
######################################

## Compute some metrics you're interested in and see how they differ for
## students who pass the subway project vs. students who don't. A good
## starting point would be the metrics we looked at earlier (minutes spent
## in the classroom, lessons completed, and days visited).

## Making Histograms

In [ ]:
######################################
#                 13                 #
######################################

## Make histograms of the three metrics we looked at earlier for both
## students who passed the subway project and students who didn't. You
## might also want to make histograms of any other metrics you examined.

## Improving Plots and Sharing Findings

In [ ]:
######################################
#                 14                 #
######################################

## Make a more polished version of at least one of your visualizations
## from earlier. Try importing the seaborn library to make the visualization
## look better, adding axis labels and a title, and changing one or more
## arguments to the hist() function.